In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 2.9 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import timeit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import streamlit as st
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, matthews_corrcoef
from imblearn.over_sampling import SMOTE

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Bidirectional, Attention, Conv1D, GlobalMaxPooling1D, Flatten, Reshape

st.title('Credit Card Fraud Detection!')

# Load and cache data
df = st.cache(pd.read_csv)(r"creditcard.csv")

# Display the dataframe details
if st.sidebar.checkbox('Show what the dataframe looks like'):
    st.write(df.head(100))
    st.write('Shape of the dataframe: ', df.shape)
    st.write('Data description:', df.describe())

# Separate fraud and valid transactions
fraud = df[df.Class == 1]
valid = df[df.Class == 0]
outlier_percentage = (df.Class.value_counts()[1] / df.Class.value_counts()[0]) * 100

if st.sidebar.checkbox('Show fraud and valid transaction details'):
    st.write('Fraudulent transactions: {:.3f}%'.format(outlier_percentage))
    st.write('Fraud Cases:', len(fraud))
    st.write('Valid Cases:', len(valid))

# Prepare features (X) and labels (y)
X = df.drop(['Class'], axis=1)
y = df.Class

# Split the data into training and testing sets
size = st.sidebar.slider('Test Set Size', min_value=0.2, max_value=0.4, step=0.05)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size, random_state=42)

# Apply SMOTE for handling class imbalance in training set
smt = SMOTE(random_state=42)
X_train, y_train = smt.fit_resample(X_train, y_train)

# Reshape X_train and X_test to 3D shape (samples, time_steps, features) for LSTM
X_train = X_train.values.reshape(-1, X_train.shape[1], 1)
X_test = X_test.values.reshape(-1, X_test.shape[1], 1)

# Convolutional Autoencoder
input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))
x = Conv1D(32, kernel_size=2, activation='relu', padding='same')(input_layer)
x = Conv1D(16, kernel_size=2, activation='relu', padding='same')(x)
encoded = GlobalMaxPooling1D()(x)

x = Dense(X_train.shape[1])(encoded)
decoded = Reshape((X_train.shape[1], 1))(x)

autoencoder = Model(inputs=input_layer, outputs=decoded)
autoencoder.compile(optimizer='adam', loss='mse')
st.write("Training Convolutional Autoencoder...")
autoencoder.fit(X_train, X_train, epochs=10, batch_size=32, validation_data=(X_test, X_test), verbose=1)

# Extract the encoded features
encoder = Model(inputs=input_layer, outputs=encoded)
X_train_encoded = encoder.predict(X_train)
X_test_encoded = encoder.predict(X_test)

# Bidirectional LSTM with Attention
def attention_3d_block(inputs):
    attention = Dense(1, activation='tanh')(inputs)
    attention = Flatten()(attention)
    attention = tf.keras.layers.Activation('softmax')(attention)
    attention = tf.keras.layers.RepeatVector(inputs.shape[2])(attention)
    attention = tf.keras.layers.Permute([2, 1])(attention)
    output_attention_mul = tf.keras.layers.multiply([inputs, attention])
    return output_attention_mul

lstm_input = Reshape((X_train_encoded.shape[1], 1))(Input(shape=(X_train_encoded.shape[1],)))
lstm_out = Bidirectional(LSTM(64, return_sequences=True))(lstm_input)
lstm_out = attention_3d_block(lstm_out)
lstm_out = Flatten()(lstm_out)
output = Dense(1, activation='sigmoid')(lstm_out)

model = Model(inputs=lstm_input, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
st.write("Training BiLSTM with Attention...")
model.fit(X_train_encoded, y_train, epochs=10, batch_size=32, validation_data=(X_test_encoded, y_test), verbose=1)

# Evaluate Model Performance
def evaluate_model(model, X_test, y_test):
    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    cm = confusion_matrix(y_test, y_pred)
    cr = classification_report(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)
    st.write("Confusion Matrix:", cm)
    st.write("Classification Report:", cr)
    st.write("Matthews Correlation Coefficient:", mcc)

# Show Evaluation
if st.sidebar.checkbox('Evaluate Model'):
    evaluate_model(model, X_test_encoded, y_test)


2024-11-10 04:17:54.579 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-10 04:17:54.590 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-10 04:17:54.599 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-10 04:17:54.605 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-10 04:17:54.617 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new
caching commands, `st.cache_data` or `st.cache_resource`. More information
[in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new caching
logic used by `st.cache_data` and `st.cache_resource`. This might lead to some problems
or unexpected behavior in certain edge cases.

2024-11-1

Epoch 1/10
6034/6034 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - loss: 4445996.0000 - val_loss: nan
Epoch 2/10
6034/6034 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 931.2023 - val_loss: nan
Epoch 3/10
6034/6034 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - loss: 121.9091 - val_loss: nan
Epoch 4/10
6034/6034 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 111.8904 - val_loss: nan
Epoch 5/10
6034/6034 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - loss: 134.3936 - val_loss: nan
Epoch 6/10
6034/6034 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 113.2008 - val_loss: nan
Epoch 7/10
6034/6034 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 66.7573 - val_loss: nan
Epoch 8/10
6034/6034 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - loss: 73.3333 - val_loss: nan
Epoch 9/10
6034/6034 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - loss: 103.2471 - val_loss: nan
Epoch 10/10
6034/6034 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - loss: 66.7144 - val_loss: nan
6034/6034 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step
756/756 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


2024-11-10 04:21:29.785 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-10 04:21:29.787 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-10 04:21:29.791 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-10 04:21:29.793 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Epoch 1/10
6034/6034 ━━━━━━━━━━━━━━━━━━━━ 119s 19ms/step - accuracy: 0.9083 - loss: 0.2543 - val_accuracy: 0.9340 - val_loss: nan
Epoch 2/10
6034/6034 ━━━━━━━━━━━━━━━━━━━━ 141s 19ms/step - accuracy: 0.9247 - loss: 0.2194 - val_accuracy: 0.9560 - val_loss: nan
Epoch 3/10
6034/6034 ━━━━━━━━━━━━━━━━━━━━ 119s 20ms/step - accuracy: 0.9268 - loss: 0.2131 - val_accuracy: 0.9239 - val_loss: nan
Epoch 4/10
6034/6034 ━━━━━━━━━━━━━━━━━━━━ 114s 19ms/step - accuracy: 0.9267 - loss: 0.2125 - val_accuracy: 0.9523 - val_loss: nan
Epoch 5/10
6034/6034 ━━━━━━━━━━━━━━━━━━━━ 115s 19ms/step - accuracy: 0.9279 - loss: 0.2076 - val_accuracy: 0.9397 - val_loss: nan
Epoch 6/10
6034/6034 ━━━━━━━━━━━━━━━━━━━━ 142s 19ms/step - accuracy: 0.9290 - loss: 0.2048 - val_accuracy: 0.9395 - val_loss: nan
Epoch 7/10
6034/6034 ━━━━━━━━━━━━━━━━━━━━ 141s 19ms/step - accuracy: 0.9283 - loss: 0.2031 - val_accuracy: 0.9413 - val_loss: nan
Epoch 8/10
6034/6034 ━━━━━━━━━━━━━━━━━━━━ 115s 19ms/step - accuracy: 0.9290 - loss: 0.2017

2024-11-10 04:42:07.569 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-10 04:42:07.570 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-10 04:42:07.574 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-10 04:42:07.577 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-10 04:42:07.580 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
